In [ ]:
#import libraries
import numpy as np
import pandas
from sklearn.model_selection import train_test_split
from time import time
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
import keras.utils

In [ ]:
#import data
data_small = pandas.read_csv('emails.csv')
y = data_small['Prediction']
X = data_small[data_small.columns[1:-1]]

# 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# 80-20 train-validation split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size = 0.2, random_state = 1)

In [ ]:
#preprocess

indices_to_keep=~X_training.isin([np.nan,np.inf,-np.inf]).any(1)
X_training = X_training[indices_to_keep]
y_training = y_training[indices_to_keep]

indices_to_keep=~X_validation.isin([np.nan,np.inf,-np.inf]).any(1)
X_validation = X_validation[indices_to_keep]
y_validation = y_validation[indices_to_keep]

indices_to_keep_test=~X_test.isin([np.nan,np.inf,-np.inf]).any(1)

X_test=X_test[indices_to_keep_test]
y_test=y_test[indices_to_keep_test]

In [ ]:
length_train = len(X_training)
X_training=X_training.iloc[:,:].values.reshape(length_train,3000,1)

length_validation = len(X_validation)
X_validation=X_validation.iloc[:,:].values.reshape(length_validation,3000,1)

length_t = len(X_test)
X_test = X_test.iloc[:,:].values.reshape(length_t,3000,1)

In [ ]:
y_training_n = []
for item in y_training:
    if item == 0:
        y_training_n.append(0)
    elif item == 1:
        y_training_n.append(1)
y_training_n = keras.utils.np_utils.to_categorical(y_training_n, num_classes=2)

In [ ]:
y_validation_n = []
for item in y_validation:
    if item == 0:
        y_validation_n.append(0)
    elif item == 1:
        y_validation_n.append(1)
y_validation_n = keras.utils.np_utils.to_categorical(y_validation_n, num_classes=2)

In [ ]:
y_test_n = []
for ele in y_test:
    if ele == 0:
        y_test_n.append(0)
    elif ele == 1:
        y_test_n.append(1)
y_test_n = keras.utils.np_utils.to_categorical(y_test_n, num_classes=2)

In [ ]:
#CNN
model = Sequential()
model.add(Convolution1D(32,3,activation="relu",input_shape=(3000,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Convolution1D(40,11,activation="relu",input_shape=(3000,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(2))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'RMSprop', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2998, 32)          128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1499, 32)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1489, 40)          14120     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 744, 40)          0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 29760)             0         
                                                                 
 dense (Dense)               (None, 2)                 5

In [ ]:
t0 = time()
model.fit(X_training, y_training_n, validation_data = (X_validation, y_validation_n), epochs=30)
print(time()-t0)

Epoch 1/30
104/104 [==============================] - 22s 197ms/step - loss: 0.5430 - accuracy: 0.7791 - val_loss: 0.2645 - val_accuracy: 0.8671
Epoch 2/30
104/104 [==============================] - 20s 196ms/step - loss: 0.2795 - accuracy: 0.9145 - val_loss: 0.2520 - val_accuracy: 0.8925
Epoch 3/30
104/104 [==============================] - 21s 199ms/step - loss: 0.1679 - accuracy: 0.9468 - val_loss: 0.1591 - val_accuracy: 0.9420
Epoch 4/30
104/104 [==============================] - 21s 199ms/step - loss: 0.1442 - accuracy: 0.9622 - val_loss: 0.2389 - val_accuracy: 0.9082
Epoch 5/30
104/104 [==============================] - 21s 200ms/step - loss: 0.1055 - accuracy: 0.9701 - val_loss: 0.1618 - val_accuracy: 0.9517
Epoch 6/30
104/104 [==============================] - 21s 199ms/step - loss: 0.0855 - accuracy: 0.9755 - val_loss: 0.1771 - val_accuracy: 0.9312
Epoch 7/30
104/104 [==============================] - 21s 200ms/step - loss: 0.0772 - accuracy: 0.9798 - val_loss: 0.1899 - val_ac

In [ ]:
model.predict(X_test)

array([[1.0000000e+00, 1.9631586e-19],
       [8.2105299e-07, 9.9999946e-01],
       [1.0000000e+00, 2.7458317e-19],
       ...,
       [1.0000000e+00, 3.0970793e-10],
       [1.0000000e+00, 8.6037929e-29],
       [1.0000000e+00, 5.5833426e-15]], dtype=float32)

In [ ]:
model.evaluate(X_test, y_test_n)

33/33 [==============================] - 2s 53ms/step - loss: 0.3114 - accuracy: 0.9710


[0.31137093901634216, 0.9710144996643066]